<a href="https://colab.research.google.com/github/divassya/BigDataAnalysis/blob/main/SparkDF_L6_Class_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Info 
MET CS777 Big Data Analytics

Faculty - Farshid Alizadeh-Shabdiz, PhD, MBA

Student - Assiya Karatay U95161396 karatay@bu.edu 857-294-7028

In [1]:
!pip install --ignore-installed -q pyspark==3.1.2 

     |████████████████████████████████| 212.4 MB 60 kB/s 
     |████████████████████████████████| 198 kB 48.3 MB/s 


In [5]:
import os
import sys
import requests
import numpy as np
from operator import add
import re

from numpy import dot
from numpy.linalg import norm

from pyspark import SparkContext, SQLContext
from pyspark.sql import SparkSession,Row
from pyspark.sql import functions as f
from pyspark.sql.functions import udf, col,monotonically_increasing_id
from pyspark.sql.types import *

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = SparkContext.getOrCreate()

In [3]:
#### set up the Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# choose where project files will be saved
project_folder = "/content/drive/MyDrive/CS777_BigDataAnalytics/class_projects/"
# project_folder = sys.argv[2]
# change the OS to use the project folder as the working directory
os.chdir(project_folder)

print('\n Working directory was changed to ' + project_folder )


 Working directory was changed to /content/drive/MyDrive/CS777_BigDataAnalytics/class_projects/


In [6]:
def mapper(line):
  fields = line.split(',')
  return Row(ID=int(fields[0]), name=str(fields[1].encode('utf-8')), age=int(fields[2]),grade=int(fields[3]))

In [8]:
lines = sc.textFile(project_folder + 'friends_noHeader.csv')
people = lines.map(mapper)
people.first()

Row(ID=0, name="b'Will'", age=33, grade=385)

In [9]:
# Infer the schema
schemaPeople = spark.createDataFrame(people).cache()

In [10]:
# register df as a table
schemaPeople.createOrReplaceTempView('students')

In [11]:
# SQL can be run over dataframes that have been registered as a table
underGrads = spark.sql('SELECT * FROM students WHERE age >= 19 AND age<=23')

In [12]:
for underGrad in underGrads.collect():
  print(underGrad)

Row(ID=16, name="b'Weyoun'", age=22, grade=323)
Row(ID=21, name="b'Miles'", age=19, grade=268)
Row(ID=25, name="b'Ben'", age=21, grade=445)
Row(ID=26, name="b'Julian'", age=22, grade=100)
Row(ID=48, name="b'Nog'", age=20, grade=1)
Row(ID=52, name="b'Beverly'", age=19, grade=269)
Row(ID=54, name="b'Brunt'", age=19, grade=5)
Row(ID=60, name="b'Geordi'", age=20, grade=100)
Row(ID=66, name="b'Geordi'", age=21, grade=477)
Row(ID=72, name="b'Kasidy'", age=22, grade=179)
Row(ID=73, name="b'Brunt'", age=20, grade=384)
Row(ID=133, name="b'Quark'", age=19, grade=265)
Row(ID=136, name="b'Will'", age=19, grade=335)
Row(ID=144, name="b'Miles'", age=22, grade=93)
Row(ID=173, name="b'Leeta'", age=23, grade=129)
Row(ID=200, name="b'Kasidy'", age=21, grade=472)
Row(ID=201, name="b'Ezri'", age=23, grade=174)
Row(ID=206, name="b'Will'", age=21, grade=491)
Row(ID=215, name="b'Will'", age=22, grade=6)
Row(ID=225, name="b'Elim'", age=19, grade=106)
Row(ID=244, name="b'Dukat'", age=21, grade=471)
Row(ID=248,

In [14]:
friends = spark.read.format('csv')\
.options(header='true', inferSchema='true', sep=',')\
.load(project_folder + 'friends_withHeader.csv')

In [15]:
friends.show(2)

+------+--------+---+-------+
|userID|    name|age|friends|
+------+--------+---+-------+
|     0|    Will| 33|    385|
|     1|Jean-Luc| 26|      2|
+------+--------+---+-------+
only showing top 2 rows



In [16]:

friends.select(f.avg("age")).show()

+--------+
|avg(age)|
+--------+
|  43.708|
+--------+



In [19]:
friends.filter('age < 21').show(2)

+------+-----+---+-------+
|userID| name|age|friends|
+------+-----+---+-------+
|    21|Miles| 19|    268|
|    48|  Nog| 20|      1|
+------+-----+---+-------+
only showing top 2 rows



In [20]:
friends.groupBy('age').count().show()

+---+-----+
|age|count|
+---+-----+
| 31|    8|
| 65|    5|
| 53|    7|
| 34|    6|
| 28|   10|
| 26|   17|
| 27|    8|
| 44|   12|
| 22|    7|
| 47|    9|
| 52|   11|
| 40|   17|
| 20|    5|
| 57|   12|
| 54|   13|
| 48|   10|
| 19|   11|
| 64|   12|
| 41|    9|
| 43|    7|
+---+-----+
only showing top 20 rows



In [21]:
# example 3

In [38]:
mySchema = StructType([
        StructField('id', IntegerType(), True),
             StructField('name', StringType(), True),
             StructField('age', IntegerType(), True),
             StructField('grade', IntegerType(), True)
             ])

In [39]:
friendsNoHeader = spark.read.options(header='false', schema=mySchema, sep=',')\
.csv(project_folder + 'friends_noHeader.csv')
friendsNoHeader.show()

+---+--------+---+---+
|_c0|     _c1|_c2|_c3|
+---+--------+---+---+
|  0|    Will| 33|385|
|  1|Jean-Luc| 26|  2|
|  2|    Hugh| 55|221|
|  3|  Deanna| 40|465|
|  4|   Quark| 68| 21|
|  5|  Weyoun| 59|318|
|  6|  Gowron| 37|220|
|  7|    Will| 54|307|
|  8|  Jadzia| 38|380|
|  9|    Hugh| 27|181|
| 10|     Odo| 53|191|
| 11|     Ben| 57|372|
| 12|   Keiko| 54|253|
| 13|Jean-Luc| 56|444|
| 14|    Hugh| 43| 49|
| 15|     Rom| 36| 49|
| 16|  Weyoun| 22|323|
| 17|     Odo| 35| 13|
| 18|Jean-Luc| 45|455|
| 19|  Geordi| 60|246|
+---+--------+---+---+
only showing top 20 rows



In [35]:
friendsNoHeader.filter('_c2 > 12 and _c2 <20').select('_c1').show()

+-------+
|    _c1|
+-------+
|  Miles|
|Beverly|
|  Brunt|
|Beverly|
|  Dukat|
|  Quark|
|   Will|
|   Elim|
|   Will|
|   Data|
|  Keiko|
|  Quark|
|Beverly|
| Kasidy|
|    Nog|
|   Data|
|  Keiko|
|  Dukat|
| Kasidy|
+-------+



Example 4

In [41]:
ius = spark.read.format('csv')\
.options(header='true', inferSchema='true', sep=',')\
.load(project_folder + 'InstagramUserStats_withHeader.csv')

In [42]:
ius.show(2)

+------+-----+---------+---------+---------------+--------------+---------------+-------------------+
|UserId|posts|followers|following|EngagementGrade|EngagementRate|FollowersGrowth|OutsidersPercentage|
+------+-----+---------+---------+---------------+--------------+---------------+-------------------+
|     1|  702|      906|      677|         0.1891|       6.07064|        1.17786|            14.8936|
|     2|  160|      386|      713|          0.195|        6.2608|      0.0941265|            16.9811|
+------+-----+---------+---------+---------------+--------------+---------------+-------------------+
only showing top 2 rows



In [47]:
grouped_ius = ius.withColumn('postGroup',ius['posts']/100).select('postGroup', 'followers', 'following')

In [48]:
grouped_ius.printSchema()

root
 |-- postGroup: double (nullable = true)
 |-- followers: integer (nullable = true)
 |-- following: integer (nullable = true)



In [51]:
grouped_ius.groupby('postGroup').avg("followers").show()

+---------+------------------+
|postGroup|    avg(followers)|
+---------+------------------+
|     8.51| 4493.444444444444|
|     0.66| 836.9041666666667|
|     2.86|5842.2972972972975|
|     3.26|2087.8055555555557|
|     9.13| 5488.666666666667|
|     13.4|            2000.0|
|     5.86| 2255.923076923077|
|     6.96|            1334.0|
|     1.82|1279.2698412698412|
|     9.55|           4550.25|
|    147.0|          269800.0|
|     0.07| 575.7912431587177|
|      2.4|2350.4565217391305|
|     0.84| 1080.185628742515|
|     5.78|            3789.7|
|     4.19|            2149.0|
|     5.05|3283.5714285714284|
|    24.96|            1200.0|
|     8.73|1245.3333333333333|
|     9.72|            1047.0|
+---------+------------------+
only showing top 20 rows



In [ ]:
grouped_ius.groupby('postGroup').avg("followers").show()